# Collecting data
Since every "Mañanera" is trainscripted and uploaded to the México's goverment page, we're going to scrap it with BeautifulSoup

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests
import time

In [2]:
def obtener_datos(num_p):
  df_dialogos=pd.DataFrame(columns=["Fecha","Dialogo"])
  df_preguntas=pd.DataFrame(columns=["Fecha","Pregunta","Respuesta"])
  for n in range(num_p):
    html=requests.get(f"https://www.gob.mx/presidencia/es/archivo/articulos?filter_origin=archive&idiom=es&order=DESC&page={str(n+1)}")
    sopa=bs(html.text,"html5lib")
    entradas=sopa.find_all("article")
    urls_conf=[entrada.a.get("href") for entrada in entradas if "conferencia" in entrada.a.get("href")]
    fechas=[entrada.time.get("date") for entrada in entradas if "conferencia" in entrada.a.get("href")]
    j=0
    for url,fecha in zip(urls_conf,fechas):
      fecha=fechas[j]
      url=url.replace(u'\\"',u'').replace('/p','http://www.gob.mx/p')
      html=requests.get(url)
      soup=bs(html.text,"html5lib")
      conferencia=soup.find("div","article-body").text
      dialogos=conferencia.split(':')
      for i in range(len(dialogos)):
        if dialogos[i][-7:] == 'OBRADOR':
          dialogo_amlo=dialogos[i+1]
          df_dialogos=df_dialogos.append({"Fecha":fecha,"Dialogo":dialogo_amlo},ignore_index=True)
        elif (dialogos[i][-8:] == 'PREGUNTA' or dialogos[i][-12:] == 'INTERLOCUTOR' or dialogos[i][-12:] == 'INTERVENCIÓN') and dialogos[i+1][-7:] == 'OBRADOR':
          pregunta_amlo=dialogos[i+1]
          respuesta_amlo=dialogos[i+2]
          df_preguntas=df_preguntas.append({"Fecha":fecha,"Pregunta":pregunta_amlo,"Respuesta":respuesta_amlo},ignore_index=True)
      time.sleep(3)
      j+=1
  return df_preguntas,df_dialogos

In [3]:
df_preguntas,df_dialogos=obtener_datos(10)
print(df_preguntas.tail())
print(df_dialogos.tail())

             Fecha                                           Pregunta  \
1042  \"2022-04-28   ¿Qué temas va a tratar con el presidente Bide...   
1043  \"2022-04-28   ¿Va a subir video?\n\nPRESIDENTE ANDRÉS MANUE...   
1044  \"2022-04-28   ¿El lunes va a Ciudad Sahagún? \n\nPRESIDENTE...   
1045  \"2022-04-28   Después de la mañanera. \n\nPRESIDENTE ANDRÉS...   
1046  \"2022-04-28   ¿Es privado?\n\nPRESIDENTE ANDRÉS MANUEL LÓPE...   

                                              Respuesta  
1042   Les vamos a informar después de la llamada.\n...  
1043   Sí, es muy probable que les informemos. No, n...  
1044   Y el lunes vamos a Ciudad Sahagún, vamos a la...  
1045           Después de la mañanera. \n\nINTERVENCIÓN  
1046   Sí, es cerrado, porque también hay elecciones...  
             Fecha                                            Dialogo
1861  \"2022-04-28   Les vamos a informar después de la llamada.\n...
1862  \"2022-04-28   Sí, es muy probable que les informemos. No, n...
186

In [4]:
df_preguntas.to_csv("preguntas.csv",index=False)
df_dialogos.to_csv("dialogos.csv",index=False)